# **Final Practical Work**

### **Code developers:**


This final project tries to obtain the page rank values of the Wikipedia pages. Pyspark, the Python API for Apache Spark is used, so that massive amounts of data, like the millions of wikipedia pages, can be included in the process. In particular, this assignment has been carried out in Databricks, which is a platform for working with spark, that connects with a remote cluster that handles and stores the massive amounts of data being processed, mainly in Dataframe formats. As we will see in the following sections, in order to obtain the PageRank of each Wikipedia document, a set of steps need to be followed. Firstly, the data is preprocessed and cleaned. Then, several Dataframes containing different information are created. Finally, the page rank function is coded and the results are checked.

## **1. Imports**

In [0]:
# We first need to import the packages and functions that are characteristic from spark
# and that are used for data manipulation

import pandas as pd # library that deals with data (not in spark format)
import re # library used for regular expressions (will be used for extracting the links in the documents)

# We now proceed to import all the pyspark libraries
from pyspark.sql.types import *
from pyspark.sql.types import ArrayType, StringType,LongType
from pyspark.sql.functions import size, explode, collect_list 
from pyspark.sql.functions import *

## **2. Reading the data**

In [0]:
# The dataset that we will use in this project is the wikipedia documents that can be found in the web
# Databricks has a pickle file with the latter information 
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
wikipediaDF = spark.read.parquet("dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet")

In [0]:
# Select a random subset of the data for efficiency purposes and save it in cache
# When we save in cache a DF, eveytime it is used in the code, it does not have
# to be computed again, but instead it is accessible in the cache memory
PartialWikipediaDF = wikipediaDF.sample(fraction=0.01,seed=0).cache()

## **3. Extracting the links in a doc**

In [0]:
# This function receives a string (wikipedia text), and outputs a list of strings containing
# the titles of the documents it points to (output links)

def parse_titles(document_body):
    """
    Input: text of a wikipedia document
    Output: list of strings containing the titles of the documents it points to
    
    Regular expressions are used in order to extract the information within the text
    that is inside "[[]]". For example "[[HELLO]]" will capture "HELLO".
    """
        
    titles = re.findall(r'\[\[(.+?)\]\]',document_body.lower())

    if len(titles) == 0:
        return []

    return list(set([title.lower() for title in titles])) # to remove duplicates and make all lowercase

In [0]:
# In order to use the parse_links function with spark DFs, we need to create a user defined function
parse_titles_udf = udf(parse_titles, ArrayType(StringType()))

In [0]:
# We create a new column in the DF, "links" that contain a list with the titles of the 
# documents that are pointed to, from the respective id Wikipedia page
TempForwardDF = PartialWikipediaDF.select("title", "id", parse_titles_udf("text").alias("links"))

## **4. Mapping titles to IDs**

In [0]:
# This function extracts the IDs corresponding to the list of titles extracted in 
# parse_links

def titles_to_id(links, data_titles):
    """
    Input: 
        - Links: list with titles of output documents
        - titleidPDF: pandas DF containing the mapping between titles and IDs of wikipedia documents
    Output:
        - List of IDs of the input titles 
    
    """
    if len(links) == 0:
        return []
    
    return list(set(data_titles[data_titles.title.isin(links)].id.to_list())) # to remove duplicates

In [0]:
# Create a DF with only id and title information

title_id_PDF = TempForwardDF.select(lower("title").alias("title"),"id").toPandas()

In [0]:
# In order to use the titles2id function with spark DFs, we need to create a user defined function
titles_to_id_UDF = udf(lambda x: titles_to_id(x, title_id_PDF), ArrayType(LongType(),False))

In [0]:
# We apply the previous function to the output titles of a doc, to obtain their IDs
# Those will be stored in the "links" column
ForwardDF = TempForwardDF.select("id", "title", titles_to_id_UDF("links").alias("links")).cache()

## **5. Counting number of output links**

In [0]:
# We will now create a new variable, count_output_links, computed as the length of the list of "links" column
OutgoingsLinksCountersDF = ForwardDF.select("id", "title", "links", size("links").alias("count_output_links"))
# The most efficient way of obtaining the counter of links is by using the pyspark function size

In [0]:
OutgoingsLinksCountersPDF = OutgoingsLinksCountersDF.toPandas()

## **6. Constructing reverse table with input links**

In [0]:
# This function aims to extract the input links of a document
# by using the output links information. We use the information that 
# link x is an output of document y to extract that y is and input to x

def input_link(document_id, links):
    """
    Input: 
        - id: id of the document
        - links: list of output IDs
    Output:
        - Reversed list containing output id - original id
    """
    if len(links) == 0:
        return []
    
    return [(link, document_id) for link in links]

In [0]:
# We will now apply the previous function to the DF that contains the output links
# and we will extract the input links

ForwardRDD = ForwardDF.rdd

ReverseRDD=(ForwardRDD
            .flatMap(lambda r: input_link(r.id, r.links)) # We just need these two columns of the original DF
            .groupByKey()
            .map(lambda r: (r[0], list(r[1]), [int(OutgoingsLinksCountersPDF.loc[OutgoingsLinksCountersPDF['id']==s, 'count_output_links'].values[0]) for s in list(r[1])] )) 
            )

reverseDF = spark.createDataFrame(ReverseRDD,["id", "links", "OutgoingsLinksCountersPDF"]) 
# This new Dataframe will contain the input links and 
# output links of each of the Wikipedia documents. In addition, 
# the variable OutgoingsLinksCountersPDF shows the total amount of output links 

In [0]:
# case when id not in reverseDF because it does not have incoming links
# The id is added to the reverseDF, but the rest of the columns are empty
reversePDF = reverseDF.toPandas()
forwardPDF = ForwardDF.toPandas()

for i in forwardPDF['id']:
    if i not in reversePDF['id'].values:
        reversePDF.loc[reversePDF.shape[0]]= [i, [], []]


## **7. Page rank**

### **7.1 Initial Page Rank**

In [0]:
# we will create a new column in the reverseDF to store the page rank of each id
N = len(reversePDF)
damping_factor = 0.85
reversePDF['PageRank'] = float(damping_factor / N)

### **7.2 Updating the page rank**

In [0]:
# We create broadcast variables that can be used everywhere, including functions
d = sc.broadcast(damping_factor)
broadcast_count_total = sc.broadcast(N)
broadcast_count_linksPDF = sc.broadcast(OutgoingsLinksCountersPDF) 

In [0]:
def convergence(document_id, prev_PageRank, new_PageRank, threshold):
    """
    Input: 
        - id_: id of the document
        - prev_PageRank: PageRank value of the doc, of the previous iteration
        - new_PageRank: PageRank value of the doc, of the current iteration
        - Threshold: convergence threshold
    """
    
    previous_value = prev_PageRank.loc[prev_PageRank['id'] == document_id, 'PageRank'].values[0]
    new_value = new_PageRank.loc[new_PageRank['id'] == document_id, 'PageRank'].values[0]
    
    return 1 if (previous_value - new_value) < threshold else 0
    

In [0]:
condition = False
threshold = 0.00001

iterations = 20     
        
reverseCompleteDF = sqlContext.createDataFrame(reversePDF)
reverseCompletePDF = reverseCompleteDF.toPandas()
reverseCompleteDF_RDD = reverseCompleteDF.rdd
NewPageRankDF = reverseCompleteDF.select("id", 'links', 'OutgoingsLinksCountersPDF', "PageRank")

# We will exit the for loop if the PageRank has convergenced or if the number of iterations has reached the variable iterations
for i in range(iterations):
    # we need to calculate the share, that is used. 
    # If the document has no output links, we redistribute its rank equally among the other pages in the graph
    share = 0
    N = broadcast_count_total.value
    
    for idx in range(len(reverseCompletePDF)):
        id_link = reverseCompletePDF.loc[idx, 'id']
        num_links = broadcast_count_linksPDF.value

        if (num_links.loc[num_links['id'] == id_link, 'count_output_links'].values[0]) == 0: #If document is a dangling node
            page_rank = reverseCompletePDF.loc[idx, 'PageRank']
            share = page_rank/N 
    
    NewPageRankPDF = NewPageRankDF.toPandas()
    for idx in range(len(reverseCompletePDF)): # this for loop updates the PageRank value of every document by using the formula
        num_links = reverseCompletePDF.loc[idx, 'OutgoingsLinksCountersPDF']
        list_of_ids = reverseCompletePDF.loc[idx, 'links']
        
        new_rank = share #new_rank will have the initial value of the share, the values of the nodes that do not have output links
        if len(list_of_ids) != 0:
            for l in range(len(list_of_ids)):
                new_rank += NewPageRankPDF.loc[NewPageRankPDF['id'] == list_of_ids[l], 'PageRank'].values[0]/num_links[l]

        new_page_rank = (1-d.value)/N + d.value*new_rank #We apply a damping factor to avoid closed loops problems
        NewPageRankPDF.loc[idx, 'PageRank'] =  float(new_page_rank)

    NewPageRankDF = sqlContext.createDataFrame(NewPageRankPDF)
    #to check the condition we have created a new function (instead of using a for that would have been less efficient)
    
    # We create a user defined function to check whether the page rank values have converged or not
    udf_check_condition =  udf(lambda l: convergence(l, reverseCompletePDF, NewPageRankPDF, threshold), FloatType())
    Threshold_checkDF = reverseCompleteDF.select("id", udf_check_condition("id").alias("Condition")) #Computing convergence
    Threshold_checkPDF = Threshold_checkDF.toPandas()
    
    reverseCompleteDF_RDD = NewPageRankDF.rdd
    reverseCompletePDF = NewPageRankPDF
    
    N = broadcast_count_total.value
    iterations += 1
                                                                         
    # In this if statement, the two exit conditions are checked
    if Threshold_checkPDF['Condition'].sum() == N: #Checking exit condition
        break
    


### **Validations**

In [0]:
 # In order to make sure that the results are correct, we check the number of unique
 # Page Rank values, and observe that indeed are a lot.
len(NewPageRankPDF['PageRank'].unique())

Out[20]: 3

In [0]:
# In addition, some final values are showed. 
NewPageRankDF.select('PageRank').distinct().show(10, False)

+---------------------+
|PageRank             |
+---------------------+
|2.6080153003564294E-4|
|4.824828305659394E-4 |
|0.001728565684882239 |
+---------------------+



## **8. Conclusions**

Throughout this final practical work, had to compute the page rank for Wikipedia's webpages using all that we have learnt during the course about Spark. In order to achieve this, we followed some previous steps:

1. For each webpage, we extracted the outgoing links mentioned within the text. To do this, we made use of regular expressions because these links are always found inside "[]".

2. We then created a dataframe containing the essential information: id, title and outgoing links.

3. The next step was to convert the extracted links into ids (the extracted links could be read as titles and we wanted in them as ids.)

4. We also included a column with the number of ourgoing links for each document and created a new DF for input links for each document.

5. Finally, we implemented the Page Rank algorithm.

The most valuable aspect of this project was gaining insight into the challenges of managing and processing large amounts of data. We made efforts to optimize resources, such as caching data structures like Dataframes, to improve computational efficiency and reduce the time required for the code to run. However, we discovered that certain operations such as toPandas() are necessary but can be costly in terms of computation.